#### TensorFlow

In [1]:
#import
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

sentence = (
    'Sega is a Japane video game develer and puisher headquarted in Shigawa, Tokyo.'
    'Sega released its next console, the Sega Genesis, in 1988; it found success outside Japan starting with Sonic the Hedgehog in 1991. In 2001, Sega stopped making consoles to become a third-party developer and publisher, and was acquired by Sammy Corporation in 2004. '
)

word_dict = {w: i for i, w in enumerate(list(set(sentence.split())))}
number_dict = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word_dict)
n_step = len(sentence.split())
n_hidden = 5

W0603 20:50:16.988708 25012 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


In [2]:
def make_batch(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i + 1)]]
        input = input + [0] * (n_step - len(input))
        target = word_dict[words[i + 1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

# Bi-LSTM Model
X = tf.placeholder(tf.float32, [None, n_step, n_class])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden * 2, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)

# outputs : [batch_size, len_seq, n_hidden], states : [batch_size, n_hidden]
outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell,lstm_bw_cell, X, dtype=tf.float32)

outputs = tf.concat([outputs[0], outputs[1]], 2) # output[0] : lstm_fw, output[1] : lstm_bw
outputs = tf.transpose(outputs, [1, 0, 2]) # [n_step, batch_size, n_hidden]
outputs = outputs[-1] # [batch_size, n_hidden]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model, 1), tf.int32)

W0603 20:50:17.142657 25012 deprecation.py:323] From <ipython-input-2-ec1a347e67dc>:22: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0603 20:50:17.144651 25012 deprecation.py:323] From <ipython-input-2-ec1a347e67dc>:26: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
W0603 20:50:17.144651 25012 deprecation.py:323] From C:\Users\Kunal\Anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0

In [3]:
# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sentence)

for epoch in range(10000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

predict =  sess.run([prediction], feed_dict={X: input_batch})
print(sentence)
print([number_dict[n] for n in [pre for pre in predict[0]]])

Epoch: 1000 cost = 2.030356
Epoch: 2000 cost = 1.541773
Epoch: 3000 cost = 1.230254
Epoch: 4000 cost = 0.995753
Epoch: 5000 cost = 0.784364
Epoch: 6000 cost = 0.868614
Epoch: 7000 cost = 0.561370
Epoch: 8000 cost = 0.453777
Epoch: 9000 cost = 0.360871
Epoch: 10000 cost = 0.302328
Sega is a Japane video game develer and puisher headquarted in Shigawa, Tokyo.Sega released its next console, the Sega Genesis, in 1988; it found success outside Japan starting with Sonic the Hedgehog in 1991. In 2001, Sega stopped making consoles to become a third-party developer and publisher, and was acquired by Sammy Corporation in 2004. 
['is', 'a', 'Japane', 'video', 'game', 'develer', 'and', 'puisher', 'headquarted', 'in', 'Shigawa,', 'Tokyo.Sega', 'released', 'its', 'Sega', 'console,', 'the', 'Sega', 'Genesis,', 'in', 'Genesis,', 'it', 'success', 'success', 'Japan', 'Japan', 'starting', 'with', 'Sonic', 'the', 'Hedgehog', 'in', '1991.', 'In', '2001,', 'Sega', 'stopped', 'making', 'in', 'to', 'become', 

#### Pytorch

In [4]:
#import
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor

sentence = (
    'Sega is a Japane video game develer and puisher headquarted in Shigawa, Tokyo.'
    'Sega released its next coole, the Sega Genes, in 1988; it found succe otside Japan startg with Sonic the Hedgog in 1991. In 2001, Sega stopped making consoles to become a third-party developer and publisher, and was acquired by Sammy Corporation in 2004. '
)

word_dict = {w: i for i, w in enumerate(list(set(sentence.split())))}
number_dict = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word_dict)
max_len = len(sentence.split())
n_hidden = 5

In [5]:
def make_batch(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i + 1)]]
        input = input + [0] * (max_len - len(input))
        target = word_dict[words[i + 1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))

class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        self.W = nn.Parameter(torch.randn([n_hidden * 2, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, X):
        input = X.transpose(0, 1)  # input : [n_step, batch_size, n_class]

        hidden_state = Variable(torch.zeros(1*2, len(X), n_hidden))   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1*2, len(X), n_hidden))     # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = torch.mm(outputs, self.W) + self.b  # model : [batch_size, n_class]
        return model

input_batch, target_batch = make_batch(sentence)

model = BiLSTM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training
for epoch in range(10000):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([number_dict[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 3.488153
Epoch: 2000 cost = 2.876433
Epoch: 3000 cost = 2.657035
Epoch: 4000 cost = 2.370752
Epoch: 5000 cost = 2.056541
Epoch: 6000 cost = 1.816891
Epoch: 7000 cost = 1.607640
Epoch: 8000 cost = 1.443380
Epoch: 9000 cost = 1.330207
Epoch: 10000 cost = 1.213806
Sega is a Japane video game develer and puisher headquarted in Shigawa, Tokyo.Sega released its next coole, the Sega Genes, in 1988; it found succe otside Japan startg with Sonic the Hedgog in 1991. In 2001, Sega stopped making consoles to become a third-party developer and publisher, and was acquired by Sammy Corporation in 2004. 
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'and', 'coole,', 'in', 'coole,', 'in', 'in', 'in', 'in', 'the', 'it', 'succe', 'succe', 'otside', 'Japan', 'startg', 'with', 'Sonic', 'the', 'Sonic', 'in', '1991.', 'In', '2001,', 'Sega', 'stopped', 'making', 'consoles', 'to', 'become', 'a', 'third-party', 'developer', 'and', 'publisher,', 'and', 'was', 'acquired', 'by', '